In [1]:
from sacla import *


In [2]:
# %load imports.py
import matplotlib.pyplot as plt
%matplotlib inline 
import numpy as np
import pandas as pd
pd.options.display.max_rows = 999
import scipy as sp
import scipy.ndimage as snd
import scipy.signal as ss
import skimage.morphology
import mkl_fft
import fast_histogram
import sys, os, datetime, shutil, glob, itertools, collections, pathlib
import h5py
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib as mpl
%load_ext line_profiler
%load_ext memory_profiler
from sacla import *
from accum import *
import idi.reconstruction as recon


def browse(obj,depth=0):
    try:
        if type(obj)==h5py._hl.dataset.Dataset:
            print(obj.shape,obj.dtype,end='')
            if np.product(obj.shape)<10: print(np.array(obj), end='')
        else:
            for k in obj.keys():
                print('\n',depth*'   ',k,end='')
                browse(obj[k],depth+1)
            print()
    except Exception as e:
        print(e)

def zoom(cor):
        cor[cor.shape[0]//2,cor.shape[1]//2]=np.nan
        plt.matshow(cor[cor.shape[0]//2-n:cor.shape[0]//2+n,cor.shape[1]//2-n:cor.shape[1]//2+n])
    


/home/gorkhover/anaconda3/envs/zimmf4/lib/python3.8/site-packages/idi/reconstruction/__init__.py:60: UserWarning: no cuda available
  _w.warn("no cuda available")


In [44]:
l=log('sacla2020.csv')

In [46]:
l[1:50]

,Run start,last run,Name,Target,Target Material,Pre-Run Comment,Post-Run Comment,Filter Oktal,Filter Dual,Distance_Dual,Distance_Oktal,focus y,p1,p2_stats,p2_thres1000,p2_thres1500,p2_thres2000,Att,Defocus
,#,#,Name,ID,Material,Text,Text,Material,Material,cm,cm,um,Path,Path,Path,Path,Path,mm Silicon,um
1,918280,918289,background,2,None,Background recording. Motor is still scanning.,NaN,None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,918308,918348,Cu20um_focus,3,Cu20um,NaN,On the dual there are photons with ~160ADU and...,None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,918349,918368,Cu20um_defocus-500,3,Cu20um,NaN,NaN,None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,918369,918413,Fe500nm_focus,4,Fe500nm,Try 0.5um iron foil. get back into focus. Also...,"Diffraction visible, see SLACK. Live analysis ...",None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,918414,918456,Fe500nm_defocus-200,4,Fe500nm,moved out of focus (-200um),foil entirely shot\r\n,None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,918457,918467,dark,none,none,dark runs for detector. no FEL,NaN,None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,918468,918510,Fe10um_focus,6,Fe10um,NaN,NaN,None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,918511,918551,Fe10um_defocus-400,6,Fe10um,moved out of focus (-400um),NaN,None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,918552,918593,Fe500nm_focus_2,7,Fe500nm,"smaller window, back in focus. will afterwards...",NaN,None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## foils ##

In [5]:
import glob
from sacla import *
outpath='/work/gorkhover/foils/p2/final'
maskfile='/work/gorkhover/foils/foil_mask_assembledtiles_final.h5'

#!mkdir -p $outpath/cont2
!mkdir -p $outpath

filenames=glob.glob('/UserData/gorkhover/TAIS2020/foils/p1/final/*.h5')
cmds=[]
for filename in filenames:
    name=os.path.splitext(os.path.basename(filename))[0][:-3]
    if '500nm' in name:
        dualsettings='+maxphotons 2 +scatterphotons 0 +skipphotons 5'
        octalsettings='+maxphotons 2 +scatterphotons 1 +skipphotons 15'
    elif 'nano' in filename:
        dualsettings='+maxphotons 2 +scatterphotons 0 +skipphotons 5'
        octalsettings='+maxphotons 2 +scatterphotons 1 +skipphotons 15'
    else:
        dualsettings='+maxphotons 3 +scatterphotons 0 +skipphotons 5'
        octalsettings='+maxphotons 3 +scatterphotons 0 +skipphotons 15'
    if 'Fe500' in name:
        photonsettings='[[3165, 7766, 11233, 14187, 17755, 20560, 24441, 26695], [0, 1, 0, 2, 1, 3, 2, 4, 0]]'
    elif 'Fe10um' in name:
        photonsettings='[[3166, 9456, 15732, 22044, 28493, 34665], [0, 1, 2, 3, 4, 5, 0]]'
    elif 'Cu' in name:
        photonsettings='[[3990, 8613, 9704, 11822, 16414, 17449, 18081, 19660, 24251, 25244, 26177, 27507, 32099, 33084, 34043, 34427, 34753, 35508, 39944, 40976, 41731, 43205, 45745, 46845, 49335], [0, 1, 0, 1, 2, 1, 0, 2, 3, 2, 1, 3, 4, 3, 2, 1, 3, 4, 5, 4, 3, 2, 6, 4, 5, 0]]' 
    else:
        raise NotImplementedError()
    
    
    
    for thresn,thresp, mode in (
        ('1000',' --deltaehigh 1000 --deltaelow 1000 ', 'intv'),
        ('1500',' --deltaehigh 1500 --deltaelow 1500 ', 'intv'),
        ('1000',' --deltaehigh 1000 --deltaelow 1000 ', 'disc'),
        ('1500',' --deltaehigh 1500 --deltaelow 1500 ', 'disc'),
        ('1500',f'--deltaehigh 1500 --deltaelow 1500 --photonsettings {photonsettings}', 'discSettings'),
        ('1500',' --deltaehigh 1500 --deltaelow 1500', 'cont'),
    ):
        for detname, detstr in (
            ('dual',f'--det dual dual both {dualsettings} --det octal octal stat {octalsettings}'),
            #('octal',f'--det dual stat {dualsettings} --det octal both {octalsettings}')
        ):
           
                outputfile=os.path.join(outpath,'',name+f'_thres{thresn}_{mode}_{detname}_p2.h5')
                if os.path.exists(outputfile):
                    print('skipping existing',outputfile)
                    continue
                #cmd=f"python ~/zimmf/phase23.py {filename} {outputfile} --det octal stat --det dual stat --name {name} --nimages 30000 --cont {thresp} --name {name} --maxphotons {maxphotons} --maskfile /UserData/gorkhover/TAIS2020/nano/nano_mask1.h5"
                cmd=f"python ~/zimmf/msc/sacla/phase2.py {filename} {outputfile} --name {name} --nimages 30000 --{mode} --nonpixelmeannormalised --singleshotcorr --pearson {thresp}  --name {name} --maskfile {maskfile} {detstr} --pulsethres 450"

                cmds.append((f'{name}',cmd))
    

skipping existing /work/gorkhover/foils/p2/final/Cu5um_focus_thres1000_intv_dual_p2.h5
skipping existing /work/gorkhover/foils/p2/final/Cu5um_focus_thres1500_intv_dual_p2.h5
skipping existing /work/gorkhover/foils/p2/final/Cu5um_focus_thres1000_disc_dual_p2.h5
skipping existing /work/gorkhover/foils/p2/final/Cu5um_focus_thres1500_disc_dual_p2.h5
skipping existing /work/gorkhover/foils/p2/final/Cu5um_focus_thres1500_discSettings_dual_p2.h5
skipping existing /work/gorkhover/foils/p2/final/Cu5um_focus_thres1500_cont_dual_p2.h5
skipping existing /work/gorkhover/foils/p2/final/Cu20um_defocus-500_thres1000_intv_dual_p2.h5
skipping existing /work/gorkhover/foils/p2/final/Cu20um_defocus-500_thres1500_intv_dual_p2.h5
skipping existing /work/gorkhover/foils/p2/final/Cu20um_defocus-500_thres1000_disc_dual_p2.h5
skipping existing /work/gorkhover/foils/p2/final/Cu20um_defocus-500_thres1500_disc_dual_p2.h5
skipping existing /work/gorkhover/foils/p2/final/Cu20um_defocus-500_thres1500_discSettings_dua

In [7]:
for name,cmd in cmds:
        print(name,cmd)
        print(qsub(['eval "$(conda shell.bash hook)"','conda activate ~/anaconda3/envs/zimmf4',cmd],jobname=f"p2f3_{name}"))

Cu5um_defocus-250 python ~/zimmf/msc/sacla/phase2.py /UserData/gorkhover/TAIS2020/foils/p1/final/Cu5um_defocus-250_p1.h5 /work/gorkhover/foils/p2/final/Cu5um_defocus-250_thres1000_intv_dual_p2.h5 --name Cu5um_defocus-250 --nimages 30000 --intv --nonpixelmeannormalised --singleshotcorr --pearson  --deltaehigh 1000 --deltaelow 1000   --name Cu5um_defocus-250 --maskfile /work/gorkhover/foils/foil_mask_assembledtiles_final.h5 --det dual dual both +maxphotons 3 +scatterphotons 0 +skipphotons 5 --det octal octal stat +maxphotons 3 +scatterphotons 0 +skipphotons 15 --pulsethres 450
(0, '47399.fep01\n')
Cu5um_defocus-250 python ~/zimmf/msc/sacla/phase2.py /UserData/gorkhover/TAIS2020/foils/p1/final/Cu5um_defocus-250_p1.h5 /work/gorkhover/foils/p2/final/Cu5um_defocus-250_thres1500_intv_dual_p2.h5 --name Cu5um_defocus-250 --nimages 30000 --intv --nonpixelmeannormalised --singleshotcorr --pearson  --deltaehigh 1500 --deltaelow 1500   --name Cu5um_defocus-250 --maskfile /work/gorkhover/foils/foil_

In [117]:
import glob
from sacla import *
outpath='/work/gorkhover/foils/p2/final-more'
maskfile='/work/gorkhover/foils/foil_mask_assembledtiles_final.h5'

#!mkdir -p $outpath/cont2
!mkdir -p $outpath

filenames=glob.glob('/UserData/gorkhover/TAIS2020/foils/p1/final/*.h5')
cmds=[]
for filename in filenames:
    name=os.path.splitext(os.path.basename(filename))[0][:-3]
    if '500nm' in name:
        dualsettings='+maxphotons 2 +scatterphotons 0 +skipphotons 5'
        octalsettings='+maxphotons 2 +scatterphotons 1 +skipphotons 15'
    elif 'nano' in filename:
        dualsettings='+maxphotons 2 +scatterphotons 0 +skipphotons 5'
        octalsettings='+maxphotons 2 +scatterphotons 1 +skipphotons 15'
    else:
        dualsettings='+maxphotons 3 +scatterphotons 0 +skipphotons 5'
        octalsettings='+maxphotons 3 +scatterphotons 0 +skipphotons 15'
    if 'Fe500' in name:
        photonsettings='[[3165, 7766, 11233, 14187, 17755, 20560, 24441, 26695], [0, 1, 0, 2, 1, 3, 2, 4, 0]]'
    elif 'Fe10um' in name:
        photonsettings='[[3166, 9456, 15732, 22044, 28493, 34665], [0, 1, 2, 3, 4, 5, 0]]'
    elif 'Cu' in name:
        photonsettings='[[3990, 8613, 9704, 11822, 16414, 17449, 18081, 19660, 24251, 25244, 26177, 27507, 32099, 33084, 34043, 34427, 34753, 35508, 39944, 40976, 41731, 43205, 45745, 46845, 49335], [0, 1, 0, 1, 2, 1, 0, 2, 3, 2, 1, 3, 4, 3, 2, 1, 3, 4, 5, 4, 3, 2, 6, 4, 5, 0]]' 
    else:
        raise NotImplementedError()
    
    
    
    for thresn,thresp, mode in (
        ('1500',' --deltaehigh 1500 --deltaelow 1500 ', 'disc'),
    ):
        for detname, detstr in (
            ('dual',f'--det octal octal stat {octalsettings} --det dual dual both {dualsettings}'),
            #('octal',f'--det dual stat {dualsettings} --det octal both {octalsettings}')
        ):
                if not 'Cu5um_focus' in name or 'filter' in name: continue
                outputfile=os.path.join(outpath,'',name+f'_thres{thresn}_{mode}_{detname}_p2.h5')
                if os.path.exists(outputfile):
                    print('skipping existing',outputfile)
                    continue
                #cmd=f"python ~/zimmf/phase23.py {filename} {outputfile} --det octal stat --det dual stat --name {name} --nimages 30000 --cont {thresp} --name {name} --maxphotons {maxphotons} --maskfile /UserData/gorkhover/TAIS2020/nano/nano_mask1.h5"
                cmd=f"MKL_THREADING_LAYER=GNU python ~/zimmf/msc/sacla/phase2-morethreads.py {filename} {outputfile} --name {name} --nimages 40000 --{mode}  --singleshotcorr  {thresp}  --name {name} --maskfile {maskfile} {detstr} --pulsethres 350"

                cmds.append((f'{name}',cmd))
    

In [119]:
for name,cmd in cmds:
        print(name,cmd)
        print(qsub(['eval "$(conda shell.bash hook)"','conda activate ~/anaconda3/envs/zimmf4',cmd],jobname=f"p2f4_{name}",threads=24))

Cu5um_focus MKL_THREADING_LAYER=GNU python ~/zimmf/msc/sacla/phase2-morethreads.py /UserData/gorkhover/TAIS2020/foils/p1/final/Cu5um_focus_p1.h5 /work/gorkhover/foils/p2/final-more/Cu5um_focus_thres1500_disc_dual_p2.h5 --name Cu5um_focus --nimages 40000 --disc  --singleshotcorr   --deltaehigh 1500 --deltaelow 1500   --name Cu5um_focus --maskfile /work/gorkhover/foils/foil_mask_assembledtiles_final.h5 --det octal octal stat +maxphotons 3 +scatterphotons 0 +skipphotons 15 --det dual dual both +maxphotons 3 +scatterphotons 0 +skipphotons 5 --pulsethres 350
(0, '64114.fep01\n')


## nano ##

In [38]:
import glob
from sacla import *
outpath='/work/gorkhover/nano/p2/final/'
maskfile='/work/gorkhover/nano/nano_mask_assembledtiles_final.h5'

#!mkdir -p $outpath/cont2
!mkdir -p $outpath

filenames=glob.glob('/UserData/gorkhover/TAIS2020/nano/p1/final/*.h5')
cmds=[]
      
for thresn,thresp in [
    ('1000',' --deltaehigh 1000 --deltaelow 1000 '),
    ('1500',' --deltaehigh 1500 --deltaelow 1500 '),
]:    
    for filename in filenames:
        name=os.path.splitext(os.path.basename(filename))[0][:-3]
        

        
        if '500nm' in name:
            dualsettings='+maxphotons 2 +scatterphotons 0 +skipphotons 5'
            octalsettings='+maxphotons 2 +scatterphotons 1 +skipphotons 15 +correct'
        elif 'nano' in filename:
            dualsettings='+maxphotons 2 +scatterphotons 0 +skipphotons 5'
            octalsettings='+maxphotons 2 +scatterphotons 1 +skipphotons 15 +correct'
        elif 'focus_nofilter' in filename:
            dualsettings='+maxphotons 2 +scatterphotons 0 +skipphotons 5'
            octalsettings='+maxphotons 2 +scatterphotons 1 +skipphotons 30 +correct' 
        else:
            dualsettings='+maxphotons 3 +scatterphotons 0 +skipphotons 5'
            octalsettings='+maxphotons 3 +scatterphotons 0 +skipphotons 15 +correct'



        for detn,detstr in [
            ('octal',f'--det octal octal both {octalsettings}'),
#            ('dual',f'--det octal octal stat {octalsettings} --det dual dual both {dualsettings}')
        ]:

            mode='--disc'
            outputfile=os.path.join(outpath,'',name+f'_thres{thresn}_disc_{detn}_p2n.h5')

            #cmd=f"python ~/zimmf/phase23.py {filename} {outputfile} --det octal stat --det dual stat --name {name} --nimages 30000 --cont {thresp} --name {name} --maxphotons {maxphotons} --maskfile /UserData/gorkhover/TAIS2020/nano/nano_mask1.h5"
            #cmd=f"python ~/zimmf/phase26.py {filename} {outputfile} --name {name} --nimages 30000 --{mode} --singleshotcorr {thresp} --name {name} --maskfile {maskfile} {detstr} --pulsethres 450"
            cmd=f"python ~/zimmf/msc/sacla/phase2-morethreads.py {filename} {outputfile} --name {name} --nimages 30000 {mode} {thresp} --name {name}  {detstr} --nonpixelmeannormalised --singleshotcorr --pulsethres 450 --maskfile {maskfile} --roi 100"

            cmds.append((f'{name}',cmd))
    

In [24]:
1

1

In [37]:
for name,cmd in cmds:
    if 'det dual dual both' in cmd:
        #print(name,cmd)
        #print(qsub(['eval "$(conda shell.bash hook)"','conda activate ~/anaconda3/envs/zimmf4',cmd],jobname=f"p2n_{name}"))
        pass

In [40]:
didnotwork=[]

In [41]:
for name,cmd in cmds:
    if 'det octal octal both' in cmd:
        print(name,cmd)
        ret=qsub(['eval "$(conda shell.bash hook)"','conda activate ~/anaconda3/envs/zimmf4',cmd],jobname=f"p2no2_{name}",mem=64,threads=24)
        if ret[0]!=0:
            didnotwork.append((name,cmd))
        else:
            print(ret)

copolymeronly python ~/zimmf/msc/sacla/phase2-morethreads.py /UserData/gorkhover/TAIS2020/nano/p1/final/copolymeronly_p1.h5 /work/gorkhover/nano/p2/final/copolymeronly_thres1000_disc_octal_p2n.h5 --name copolymeronly --nimages 30000 --disc  --deltaehigh 1000 --deltaelow 1000  --name copolymeronly  --det octal octal both +maxphotons 2 +scatterphotons 1 +skipphotons 15 +correct --nonpixelmeannormalised --singleshotcorr --pulsethres 450 --maskfile /work/gorkhover/nano/nano_mask_assembledtiles_final.h5 --roi 100
(0, '49226.fep01\n')
benNi40nm python ~/zimmf/msc/sacla/phase2-morethreads.py /UserData/gorkhover/TAIS2020/nano/p1/final/benNi40nm_p1.h5 /work/gorkhover/nano/p2/final/benNi40nm_thres1000_disc_octal_p2n.h5 --name benNi40nm --nimages 30000 --disc  --deltaehigh 1000 --deltaelow 1000  --name benNi40nm  --det octal octal both +maxphotons 2 +scatterphotons 1 +skipphotons 15 +correct --nonpixelmeannormalised --singleshotcorr --pulsethres 450 --maskfile /work/gorkhover/nano/nano_mask_assem

In [88]:
browse(h5py.File('/work/gorkhover/nano/nano_mask_octaltiles.h5'))


  tile0(512, 1024) bool
  tile1(512, 1024) bool
  tile2(512, 1024) bool
  tile3(512, 1024) bool
  tile4(512, 1024) bool
  tile5(512, 1024) bool
  tile6(512, 1024) bool
  tile7(512, 1024) bool


In [54]:
didnotwork4=[]
for name,cmd in didnotwork3:
    if 'det octal octal both' in cmd:
        print(name,cmd)
        ret=qsub(['eval "$(conda shell.bash hook)"','conda activate ~/anaconda3/envs/zimmf4',cmd],jobname=f"p2no2_{name}",mem=64,threads=24)
        if ret[0]!=0:
            didnotwork4.append((name,cmd))
        else:
            print(ret)

20nmcopolymer_unknown python ~/zimmf/msc/sacla/phase2-morethreads.py /UserData/gorkhover/TAIS2020/nano/p1/final/20nmcopolymer_unknown_p1.h5 /work/gorkhover/nano/p2/final/20nmcopolymer_unknown_thres1500_disc_octal_p2n.h5 --name 20nmcopolymer_unknown --nimages 30000 --disc  --deltaehigh 1500 --deltaelow 1500  --name 20nmcopolymer_unknown  --det octal octal both +maxphotons 2 +scatterphotons 1 +skipphotons 15 +correct --nonpixelmeannormalised --singleshotcorr --pulsethres 450 --maskfile /work/gorkhover/nano/nano_mask_assembledtiles_final.h5 --roi 100
(0, '53878.fep01\n')
20nmPS_focus_nofilter python ~/zimmf/msc/sacla/phase2-morethreads.py /UserData/gorkhover/TAIS2020/nano/p1/final/20nmPS_focus_nofilter_p1.h5 /work/gorkhover/nano/p2/final/20nmPS_focus_nofilter_thres1500_disc_octal_p2n.h5 --name 20nmPS_focus_nofilter --nimages 30000 --disc  --deltaehigh 1500 --deltaelow 1500  --name 20nmPS_focus_nofilter  --det octal octal both +maxphotons 2 +scatterphotons 1 +skipphotons 15 +correct --nonp

In [55]:
didnotwork4

[]

In [72]:
!ls /work/gorkhover/nano/

nano_mask_assembledtiles_final.h5  nano_mask_octaltiles.h5  res202109
nano_mask_assembledtiles.h5	   p2


In [97]:
import glob
from sacla import *
outpath='/work/gorkhover/nano/p2/final_tilewise2/'
maskfile='/work/gorkhover/nano/nano_mask_octaltiles2.h5'

#!mkdir -p $outpath/cont2
!mkdir -p $outpath

filenames=glob.glob('/UserData/gorkhover/TAIS2020/nano/p1/final/*.h5')
cmds=[]
      
for thresn,thresp in [
#    ('1000',' --deltaehigh 1000 --deltaelow 1000 '),
    ('0812',' --deltaehigh 800 --deltaelow 1200'),
]:    
    for filename in filter(lambda x: not any([s in x for s in ['ben','Spin']]),filenames):
        name=os.path.splitext(os.path.basename(filename))[0][:-3]
        
        octalsettings='+maxphotons 2 +scatterphotons 1 +skipphotons 20 +correct'

        for detn,detstr in [
            (f'octal-tilesA',' '.join([f'--det tile{t} octal both +tile {t} {octalsettings}' for t in [0,1]])),
            (f'octal-tilesB',' '.join([f'--det tile{t} octal both +tile {t} {octalsettings}' for t in [2,3]])),
            (f'octal-tilesC',' '.join([f'--det tile{t} octal both +tile {t} {octalsettings}' for t in [4,5]])),
            (f'octal-tilesD',' '.join([f'--det tile{t} octal both +tile {t} {octalsettings}' for t in [6,7]])),


        ]:
            mode='--disc'
            outputfile=os.path.join(outpath,'',name+f'_thres{thresn}_disc_{detn}_p2t.h5')

            #cmd=f"python ~/zimmf/phase23.py {filename} {outputfile} --det octal stat --det dual stat --name {name} --nimages 30000 --cont {thresp} --name {name} --maxphotons {maxphotons} --maskfile /UserData/gorkhover/TAIS2020/nano/nano_mask1.h5"
            #cmd=f"python ~/zimmf/phase26.py {filename} {outputfile} --name {name} --nimages 30000 --{mode} --singleshotcorr {thresp} --name {name} --maskfile {maskfile} {detstr} --pulsethres 450"
            cmd=f"python ~/zimmf/msc/sacla/phase2.py {filename} {outputfile} --name {name} --nimages 20000 {mode} {thresp} --name {name}  {detstr} --singleshotcorr --pulsethres 450 --maskfile {maskfile} --roi 20"

            cmds.append((f'{name}',cmd))
    

In [98]:
didnotwork4=[]
for name,cmd in cmds:
        print(name,cmd)
        ret=qsub(['eval "$(conda shell.bash hook)"','conda activate ~/anaconda3/envs/zimmf4',cmd],jobname=f"p2not9_{name}",mem=32,threads=12)
        if ret[0]!=0:
            didnotwork4.append((name,cmd))
        else:
            print(ret)

copolymeronly python ~/zimmf/msc/sacla/phase2.py /UserData/gorkhover/TAIS2020/nano/p1/final/copolymeronly_p1.h5 /work/gorkhover/nano/p2/final_tilewise2/copolymeronly_thres0812_disc_octal-tilesA_p2t.h5 --name copolymeronly --nimages 20000 --disc  --deltaehigh 800 --deltaelow 1200 --name copolymeronly  --det tile0 octal both +tile 0 +maxphotons 2 +scatterphotons 1 +skipphotons 20 +correct --det tile1 octal both +tile 1 +maxphotons 2 +scatterphotons 1 +skipphotons 20 +correct --singleshotcorr --pulsethres 450 --maskfile /work/gorkhover/nano/nano_mask_octaltiles2.h5 --roi 20
(0, '61464.fep01\n')
copolymeronly python ~/zimmf/msc/sacla/phase2.py /UserData/gorkhover/TAIS2020/nano/p1/final/copolymeronly_p1.h5 /work/gorkhover/nano/p2/final_tilewise2/copolymeronly_thres0812_disc_octal-tilesB_p2t.h5 --name copolymeronly --nimages 20000 --disc  --deltaehigh 800 --deltaelow 1200 --name copolymeronly  --det tile2 octal both +tile 2 +maxphotons 2 +scatterphotons 1 +skipphotons 20 +correct --det tile3

[]

In [77]:
!ls ~/zimmf/msc/sacla/phase2-12threads.py

/home/gorkhover/zimmf/msc/sacla/phase2-12threads.py


In [76]:
didnotwork4

[]

In [7]:
for name,cmd in cmds:
    print(name,cmd)
    print(qsub(['eval "$(conda shell.bash hook)"','conda activate zimmf3',cmd],jobname=f"p2st_{name}"))

copolymeronly python ~/zimmf/phase27.py /UserData/gorkhover/TAIS2020/nano/p1/parts/copolymeronly_p1.h5 /UserData/gorkhover/TAIS2020/nano/p2/stat_tiles/copolymeronly_thres1000_stat_p2.h5 --name copolymeronly --nimages 40000 --cont --intv --disc  --deltaehigh 1000 --deltaelow 1000  --name copolymeronly  --det dual dual stat +maxphotons 3 +scatterphotons 0 +skipphotons 5 --det octal octal stat +maxphotons 3 +scatterphotons 1 +skipphotons 15 +correct --pulsethres 450
(0, '2376251.fep02-ib\n')
benNi40nm python ~/zimmf/phase27.py /UserData/gorkhover/TAIS2020/nano/p1/parts/benNi40nm_p1.h5 /UserData/gorkhover/TAIS2020/nano/p2/stat_tiles/benNi40nm_thres1000_stat_p2.h5 --name benNi40nm --nimages 40000 --cont --intv --disc  --deltaehigh 1000 --deltaelow 1000  --name benNi40nm  --det dual dual stat +maxphotons 3 +scatterphotons 0 +skipphotons 5 --det octal octal stat +maxphotons 3 +scatterphotons 1 +skipphotons 15 +correct --pulsethres 450
(0, '2376252.fep02-ib\n')
20nmPS_+400 python ~/zimmf/phase

In [34]:
import glob
from sacla import *
outpath='/UserData/gorkhover/TAIS2020/nano/p2/stat_tiles/'
#maskfile='/UserData/gorkhover/TAIS2020/nano/nano_mask1.h5'

#!mkdir -p $outpath/cont2
!mkdir -p $outpath

filenames=glob.glob('/UserData/gorkhover/TAIS2020/nano/p1/final/*.h5')
cmds=[]
for filename in filenames:
    name=os.path.splitext(os.path.basename(filename))[0][:-3]
    if '500nm' in name:
        dualsettings='+maxphotons 3 +scatterphotons 0 +skipphotons 5'
        octalsettings='+maxphotons 3 +scatterphotons 1 +skipphotons 15 +correct'
    elif 'nano' in filename:
        dualsettings='+maxphotons 3 +scatterphotons 0 +skipphotons 5'
        octalsettings='+maxphotons 3 +scatterphotons 1 +skipphotons 15 +correct'
    else:
        dualsettings='+maxphotons 3 +scatterphotons 0 +skipphotons 5'
        octalsettings='+maxphotons 3 +scatterphotons 0 +skipphotons 15 +correct'
        
    for thresn,thresp in [
       #('1000',' --deltaehigh 1000 --deltaelow 1000 '),
        ('1500',' --deltaehigh 1500 --deltaelow 1500 '),
       # ('2000',' --deltaehigh 2000 --deltaelow 2000 '),
    ]:
        for detname, detstr in [
            #('dual',f'--det dual stat {dualsettings} --det octal both {octalsettings}'),
            ('octal',f'--det dual dual stat {dualsettings} --det octal octal stat {octalsettings}')
        ]:
            for mode in ('cont','disc'):
                outputfile=os.path.join(outpath,'',name+f'_thres{thresn}_{mode}_{detname}_p2.h5')

                #cmd=f"python ~/zimmf/phase23.py {filename} {outputfile} --det octal stat --det dual stat --name {name} --nimages 30000 --cont {thresp} --name {name} --maxphotons {maxphotons} --maskfile /UserData/gorkhover/TAIS2020/nano/nano_mask1.h5"
                #cmd=f"python ~/zimmf/phase26.py {filename} {outputfile} --name {name} --nimages 30000 --{mode} --singleshotcorr {thresp} --name {name} --maskfile {maskfile} {detstr} --pulsethres 450"
                cmd=f"python ~/zimmf/phase27.py {filename} {outputfile} --name {name} --nimages 30000 --{mode} {thresp} --name {name} --maskfile {maskfile} {detstr} --pulsethres 450"

                cmds.append((f'{name}',cmd))
    

In [36]:
for name,cmd in cmds:
    print(name,cmd)
    print(qsub(['eval "$(conda shell.bash hook)"','conda activate zimmf3',cmd],jobname=f"p2s_{name}"))

copolymeronly python ~/zimmf/phase26.py /UserData/gorkhover/TAIS2020/nano/p1/final/copolymeronly_p1.h5 /UserData/gorkhover/TAIS2020/nano/p2/final1/copolymeronly_thres1000_cont_octal_p2.h5 --name copolymeronly --nimages 30000 --cont --singleshotcorr  --deltaehigh 1000 --deltaelow 1000  --name copolymeronly --maskfile /UserData/gorkhover/TAIS2020/nano/nano_mask1.h5 --det dual both +maxphotons 2 +scatterphotons 0 +skipphotons 5 --det octal stat +maxphotons 2 +scatterphotons 1 +skipphotons 15 --pulsethres 450
(0, '2333161.fep02-ib\n')
copolymeronly python ~/zimmf/phase26.py /UserData/gorkhover/TAIS2020/nano/p1/final/copolymeronly_p1.h5 /UserData/gorkhover/TAIS2020/nano/p2/final1/copolymeronly_thres1000_disc_octal_p2.h5 --name copolymeronly --nimages 30000 --disc --singleshotcorr  --deltaehigh 1000 --deltaelow 1000  --name copolymeronly --maskfile /UserData/gorkhover/TAIS2020/nano/nano_mask1.h5 --det dual both +maxphotons 2 +scatterphotons 0 +skipphotons 5 --det octal stat +maxphotons 2 +sc

In [59]:
import glob
from sacla import *
outpath='/UserData/gorkhover/TAIS2020/nano/p2/new3/'

#!mkdir -p $outpath/cont2
!mkdir -p $outpath

filenames=glob.glob('/UserData/gorkhover/TAIS2020/nano/p1/new3/*.h5')
cmds=[]
for filename in filenames:
    name=os.path.splitext(os.path.basename(filename))[0][:-3]
    maxphotons=30 if 'nofilter' in name else 10
    #maxphotons=5
    for thresn,thresp in (
        ('1000',' --deltaehigh 1000 --deltaelow 1000 '),
        ('1500',' --deltaehigh 1500 --deltaelow 1500 '),
        ('2000',' --deltaehigh 2000 --deltaelow 2000 '),
    ):
        outputfile=os.path.join(outpath,'',name+f'_thres{thresn}_p2.h5')

        cmd=f"python ~/zimmf/phase23.py {filename} {outputfile} --det octal stat --det dual both --name {name} --nimages 30000 --cont {thresp} --name {name} --maxphotons {maxphotons} --maskfile /UserData/gorkhover/TAIS2020/nano/nano_mask1.h5"
        #cmd=f"python ~/zimmf/phase23.py {filename} {outputfile} --det dual both --name {name} --nimages 30000 --cont {thresp} --name {name} --maxphotons {maxphotons} --maskfile /home/gorkhover/work/TAIS2020/foil_mask.h5"

        cmds.append((f'{name}',cmd))
    
    #outputfile=os.path.join(outpath,"disc",name+'_p2.h5')   
    #cmd=f"python ~/zimmf/phase2.py {filename} {outputfile} --maskfile /home/gorkhover/work/TAIS2020/foil_mask.h5"
    #cmds.append((f'{name}_disc',cmd))

In [60]:
for name,cmd in cmds:
    print(name,cmd)
    print(qsub(['eval "$(conda shell.bash hook)"','conda activate zimmf3',cmd],jobname=f"p2_2_{name}"))

copolymeronly python ~/zimmf/phase23.py /UserData/gorkhover/TAIS2020/nano/p1/new3/copolymeronly_p1.h5 /UserData/gorkhover/TAIS2020/nano/p2/new3/copolymeronly_thres1000_p2.h5 --det octal stat --det dual both --name copolymeronly --nimages 30000 --cont  --deltaehigh 1000 --deltaelow 1000  --name copolymeronly --maxphotons 10 --maskfile /UserData/gorkhover/TAIS2020/nano/nano_mask1.h5
(0, '2331548.fep02-ib\n')
copolymeronly python ~/zimmf/phase23.py /UserData/gorkhover/TAIS2020/nano/p1/new3/copolymeronly_p1.h5 /UserData/gorkhover/TAIS2020/nano/p2/new3/copolymeronly_thres1500_p2.h5 --det octal stat --det dual both --name copolymeronly --nimages 30000 --cont  --deltaehigh 1500 --deltaelow 1500  --name copolymeronly --maxphotons 10 --maskfile /UserData/gorkhover/TAIS2020/nano/nano_mask1.h5
(0, '2331549.fep02-ib\n')
copolymeronly python ~/zimmf/phase23.py /UserData/gorkhover/TAIS2020/nano/p1/new3/copolymeronly_p1.h5 /UserData/gorkhover/TAIS2020/nano/p2/new3/copolymeronly_thres2000_p2.h5 --det 

In [46]:

## stats
for name,cmd in cmds:
    print(name,cmd)
    print(qsub(['eval "$(conda shell.bash hook)"','conda activate zimmf3',cmd],jobname=f"p2_15_{name}"))

copolymeronly python ~/zimmf/phase23.py /UserData/gorkhover/TAIS2020/nano/p1/new3/copolymeronly_p1.h5 /UserData/gorkhover/TAIS2020/nano/p2/stats4/copolymeronly_thres1000_p2_stats.h5 --det octal stat --det dual stat --name copolymeronly --nimages 30000 --cont  --deltaehigh 1000 --deltaelow 1000  --name copolymeronly --maxphotons 10 --maskfile /UserData/gorkhover/TAIS2020/nano/nano_mask1.h5
(0, '2331165.fep02-ib\n')
benNi40nm python ~/zimmf/phase23.py /UserData/gorkhover/TAIS2020/nano/p1/new3/benNi40nm_p1.h5 /UserData/gorkhover/TAIS2020/nano/p2/stats4/benNi40nm_thres1000_p2_stats.h5 --det octal stat --det dual stat --name benNi40nm --nimages 30000 --cont  --deltaehigh 1000 --deltaelow 1000  --name benNi40nm --maxphotons 10 --maskfile /UserData/gorkhover/TAIS2020/nano/nano_mask1.h5
(0, '2331166.fep02-ib\n')
20nmPS_+400 python ~/zimmf/phase23.py /UserData/gorkhover/TAIS2020/nano/p1/new3/20nmPS_+400_p1.h5 /UserData/gorkhover/TAIS2020/nano/p2/stats4/20nmPS_+400_thres1000_p2_stats.h5 --det oc

## crystal

In [108]:
outdir='/work/gorkhover/crystal/p2/more/'

!mkdir -p $outdir

cmds=[]
for sample,rotstr in [
    ['gaas1',"--detz 2834 --offsetx 28 --offsety 128 --rotx -0.4 --roty 0.1 --rotz 1.5"],
    ['gaas2',"--detz 2798 --offsetx 26 --offsety 116 --rotx -0.2 --roty 0.1 --rotz 1.1"]
]:
    for i in range(10):
        cmd=f'MKL_THREADING_LAYER=GNU python /home/gorkhover/zimmf/msc/sacla/phase2c.py /UserData/gorkhover/TAIS2020/crystal/p1/parts2/{sample}_p1.h5 {outdir}/{sample}_{i}.h5 --nimages 5000 --detz 2825 --name {sample} --job {i} {rotstr}' 
        cmds.append((sample,i,cmd))

In [109]:
didnotwork=[]
for name,i,cmd in cmds:
        print(name,i,cmd)
        ret=qsub(['eval "$(conda shell.bash hook)"','conda activate ~/anaconda3/envs/zimmf4',cmd],jobname=f"p2c4_{name}_{i}",mem=160,threads=24)
        if ret[0]!=0:
            didnotwork.append((name,i,cmd))
            print(ret)
        else:
            print(ret)

gaas1 0 MKL_THREADING_LAYER=GNU python /home/gorkhover/zimmf/msc/sacla/phase2c.py /UserData/gorkhover/TAIS2020/crystal/p1/parts2/gaas1_p1.h5 /work/gorkhover/crystal/p2/more//gaas1_0.h5 --nimages 5000 --detz 2825 --name gaas1 --job 0 --detz 2834 --offsetx 28 --offsety 128 --rotx -0.4 --roty 0.1 --rotz 1.5
(0, '61841.fep01\n')
gaas1 1 MKL_THREADING_LAYER=GNU python /home/gorkhover/zimmf/msc/sacla/phase2c.py /UserData/gorkhover/TAIS2020/crystal/p1/parts2/gaas1_p1.h5 /work/gorkhover/crystal/p2/more//gaas1_1.h5 --nimages 5000 --detz 2825 --name gaas1 --job 1 --detz 2834 --offsetx 28 --offsety 128 --rotx -0.4 --roty 0.1 --rotz 1.5
(0, '61842.fep01\n')
gaas1 2 MKL_THREADING_LAYER=GNU python /home/gorkhover/zimmf/msc/sacla/phase2c.py /UserData/gorkhover/TAIS2020/crystal/p1/parts2/gaas1_p1.h5 /work/gorkhover/crystal/p2/more//gaas1_2.h5 --nimages 5000 --detz 2825 --name gaas1 --job 2 --detz 2834 --offsetx 28 --offsety 128 --rotx -0.4 --roty 0.1 --rotz 1.5
(0, '61843.fep01\n')
gaas1 3 MKL_THREADI

In [ ]:
from sacla import qsub
for name,j,cmd in cmds:
        print(cmd)
        print(qsub(['eval "$(conda shell.bash hook)"','conda activate zimmf3',cmd],jobname=f"c_{name}_{j}",threads=22,mem=200,queue='smp'))

## run test ##

In [36]:
#!chmod +x phase27.py
!rm ~/work/testoutput_phase2.h5

In [2]:
2/final/Cu5um_focus_thres1000_cont_octal_p2.h5 --name Cu5um_focus --nimages 30000 --cont --singleshotcorr  --deltaehigh 1000 --deltaelow 1000  --name Cu5um_focus 

'/home/gorkhover/zimmf/msc/sacla'

In [35]:
%%bash 
source ~/.bashrc
time ./phase2.py  /home/gorkhover/UserDatagorkhover/TAIS2020/foils/p1/final/Cu5um_defocus-250_p1.h5 ~/work/testoutput_phase2.h5 --pearson  --singleshotcorr --deltaehigh 1000 --deltaelow 1000 --intv --cont --disc --discSettings --maskfile /arcv1/UserData/gorkhover/TAIS2020/nano/mask_assembledtiles.h5 --det dual dual both +maxphotons 3 +scatterphotons 0 +skipphotons 5 --det octal octal stat +maxphotons 3 +scatterphotons 0 +skipphotons 15 --pulsethres 450  --photonsettings "[[3990, 8613, 9704, 11822, 16414, 17449, 18081, 19660, 24251, 25244, 26177, 27507, 32099, 33084, 34043, 34427, 34753, 35508, 39944, 40976, 41731, 43205, 45745, 46845, 49335], [0, 1, 0, 1, 2, 1, 0, 2, 3, 2, 1, 3, 4, 3, 2, 1, 3, 4, 5, 4, 3, 2, 6, 4, 5, 0]]" --nimages 100 

Process is terminated.


In [24]:
import h5py

In [25]:
f=h5py.File("/home/gorkhover/UserDatagorkhover/TAIS2020/foils/p1/final/Cu5um_defocus-250_p1.h5")

In [27]:
browse(f)


  attenuator
     attenuator_eh_2_Al_thickness_in_meter(85017,) float32
     attenuator_eh_4_Al_thickness_in_meter(85017,) float32
     attenuator_eh_5_Si_thickness_in_meter(85017,) float32
     attenuator_eh_5_Si_uncalibrated(85017,) float32
     attenuator_oh_2_Si_thickness_in_meter(85017,) float32

  beamspectrum(85017, 1024) float32
  detectors
     dual
        tile0
           absfft0
              mean(513, 512) float32
              n(1,) int32[8519]
              std(513, 512) float32

           absfft1
              mean(1024, 257) float32
              n(1,) int32[8519]
              std(1024, 257) float32

           bg(1024, 512) float32
           data(85017, 1024, 512) int32
           edgeintensity(85017, 4) float32
           hist
              bincenters(10203,) float64
              run(40, 10203) float32
              shot(85017, 10203) float32

           intensity(85017,) float32
           max(1024, 512) float32
           mean(1024, 512) float32
           min

In [25]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import idi.reconstruction.simple as recon

#o=h5py.File('/UserData/gorkhover/TAIS2020/foils/p2/new6/cont2/Fe500nm_focus_2_p2.h5','r')
o=h5py.File('/home/gorkhover/work/testp237.h5','r')
#o=h5py.File('/UserData/gorkhover/TAIS2020/foils/p2/new13/Cu20um_defocus-500_thres1000_p2.h5','r')

In [65]:
def zoom(cor):
        cor[cor.shape[0]//2,cor.shape[1]//2]=np.nan
        plt.matshow(cor[cor.shape[0]//2-n:cor.shape[0]//2+n,cor.shape[1]//2-n:cor.shape[1]//2+n])
    

In [540]:
def browse(obj,depth=0):
    try:
        if type(obj)==h5py._hl.dataset.Dataset:
            print(obj.shape,obj.dtype,end='')
            if np.product(obj.shape)<10: print(np.array(obj), end='')
        else:
            for k in obj.keys():
                print('\n',depth*'   ',k,end='')
                browse(obj[k],depth+1)
            print()
    except Exception as e:
        print(e)



In [79]:
def get_thresholds(kalpha, deltaelow, deltaehigh, maxphotons, nscatter, scatter):
    thresholds = tuple([
        (
            float(n),
            float(s),
            n * kalpha + s * scatter - deltaelow,
            n * kalpha + s * scatter + deltaehigh,
            s*scatter
        )
        for s in range(nscatter+1) 
        for n in range(maxphotons-s+1)
    ][1:])
    return thresholds

In [43]:
get_thresholds(6000,1000,1000,2,2,11000)

((1.0, 0.0, 5000, 7000, 0),
 (2.0, 0.0, 11000, 13000, 0),
 (0.0, 1.0, 10000, 12000, 11000),
 (1.0, 1.0, 16000, 18000, 11000),
 (2.0, 1.0, 22000, 24000, 11000),
 (0.0, 2.0, 21000, 23000, 22000),
 (1.0, 2.0, 27000, 29000, 22000),
 (2.0, 2.0, 33000, 35000, 22000))

In [80]:
get_thresholds(6000,1000,1000,2,2,11000)

((1.0, 0.0, 5000, 7000, 0),
 (2.0, 0.0, 11000, 13000, 0),
 (0.0, 1.0, 10000, 12000, 11000),
 (1.0, 1.0, 16000, 18000, 11000),
 (0.0, 2.0, 21000, 23000, 22000))

In [74]:
maxphotons=2
nscatter=2
deltaelow=500
deltaehigh=500
kalpha=6000
scatter=11000 
    

((1.0, 0.0, 5500, 6500),
 (2.0, 0.0, 11500, 12500),
 (0.0, 1.0, 10500, 11500),
 (1.0, 1.0, 16500, 17500),
 (0.0, 2.0, 21500, 22500))

In [62]:
[(a,b) for b in range(5) for a in range(b) ]

[(0, 1),
 (0, 2),
 (1, 2),
 (0, 3),
 (1, 3),
 (2, 3),
 (0, 4),
 (1, 4),
 (2, 4),
 (3, 4)]

In [61]:
[x for b in a for x in b

2

In [63]:
n=None

In [64]:
s=None

In [12]:
!ls /work/gorkhover

In [40]:
ds=f['detectors/dual/tile0/data']

In [42]:
mask=np.zeros(len(ds),bool)

In [44]:
mask[:10]=True
mask[20]=True
np.sum(mask)

11

In [50]:
ds[mask.nonzero()].shape

(11, 1024, 512)

In [46]:
h5py.__version__

'3.1.0'